# Assignment 3 - Software Development

Task: do something geo-specific in a notebook!

Idea: because it is related to my work, I want to query cycling infrastructure data from OSM. From that data, I want to take one LineString, and segment it into 20m segments. Let's see if we can get this done!

Let's start this by querying cyclable infrastructure from OSM for Salzburg, using Pyrosmium:

In [1]:
import pyrosm as pyro
import numpy as np
import geopandas as gpd
import folium
import itertools
import shapely

bbox = [13.032703,47.810604,13.037059,47.819970]
bbox_center = [np.mean([bbox[1], bbox[3]]),np.mean([bbox[0], bbox[2]])]

fp = pyro.get_data("Salzburg")
osm = pyro.OSM(fp, bounding_box=bbox)
cycle_lanes = osm.get_network(network_type="cycling")

c:\Users\PC\anaconda3\envs\zgis_work\Lib\site-packages\pyrosm\networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges, nodes = prepare_geodataframe(


Now, let's find the longest cyclable linestring in here:

In [2]:
longest_lane = cycle_lanes.loc[cycle_lanes["length"] == np.max(cycle_lanes["length"])]

Okay, but where is all that? And which one is the longest lane?

In [3]:
m = folium.Map(
    bbox_center,
    zoom_start=16,
    tiles="OpenStreetMap"
)

folium.Rectangle(
    bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],
    color='#ff7800',
    fill=True,
    fill_color='#ffff00',
    fill_opacity=0.1
).add_to(m)

folium.Choropleth(
    longest_lane,
    line_weight = 3,
    line_color = "blue"
).add_to(m)

folium.Choropleth(
    cycle_lanes,
    line_weight = 1,
    line_color = "red"
).add_to(m)

m

Now, to get my cutting functionality to work, we need to do some geoinformatics voodoo: first, we "explode" the multilinestring into multiple linestrings, second, we project to some geographic CRS, third, we merge all the linestrings back together into one single object:

In [4]:
LL_linestrings = longest_lane.explode(index_parts = True)

LL_linestrings = LL_linestrings.to_crs(crs = "epsg:9272")

from shapely import ops
single_linestring_cyclelane = ops.linemerge(LL_linestrings.geometry.to_list())

To achieve my desired functionality, I found this awesome snippet on StackOverflow  (https://gis.stackexchange.com/a/414888):

In [5]:
def cut(line, distance):
    distances = np.arange(0, line.length + distance, distance)
    return list(map(ops.substring, itertools.repeat(line), distances[:-1], distances[1:]))

longest_lane_cut = cut(single_linestring_cyclelane, 20)

print([round(line.length, 4) for line in longest_lane_cut])

[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 13.711]


Now that we have the "cut" line segments, we can re-transform to WGS84 for the nice map, and plot them:

In [8]:
new_linestring_df = gpd.GeoDataFrame(geometry = longest_lane_cut, crs = "epsg:9272")
new_linestring_df = new_linestring_df.to_crs(crs="wgs84")

segment_starts = shapely.get_point(new_linestring_df.geometry, 0)

for point in segment_starts:
    folium.Marker(
        [point.y, point.x]
    ).add_to(m)
    
m